In [110]:
#Boiler plate imports
import pandas as pd
import sys
from time import time
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

#SBND imports
s0 = time()
sys.path.append('/exp/sbnd/app/users/brindenc/analyze_sbnd/pyana')
from sbnd.general import utils
from sbnd.cafclasses.slice import CAFSlice
from sbnd.cafclasses.pfp import PFP
from pyanalib.panda_helpers import multicol_concat

#Constants
from sbnd.numu.numu_constants import *
from sbnd.prism import PRISM_BINS,make_prism_plot
from sbnd.constants import *
from sbnd.volume import plot_volume_boundary

#Plotters
from sbnd.plotlibrary import makeplot
from sbnd.general import plotters

s1 = time()
print(f'SBND imports: {s1-s0:.2f} s')

%load_ext autoreload
%autoreload 2

SBND imports: 0.00 s
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
#Constants/variables
CUT_MODE = 'moon'
DATA_DIR  = '/exp/sbnd/data/users/brindenc/analyze_sbnd/numu/v09_78_04_wc_pandora'
#DATA_DIR = '/exp/sbnd/data/users/brindenc/ML/test_fcl/debug_trackid/v4'
#FNAME = 'single_processed.df'
#FNAME = 'single_cut_cosmics_processed.df'
#HDR_FNAME = 'single.df'
FNAME = f'all_processed_{CUT_MODE}.df'
#FNAME = 'all_processed_roc.df'
HDR_FNAME = 'all.df'
#FNAME = 'test_processed.df'
#HDR_FNAME = 'test.df'
NOM_POT = 0.6e20 # stats for first run
LABEL = 'SBND Work in Progress\n0.6e20 POT'
PLOT_DIR = f'Plots/reco_{plotters.day}_{CUT_MODE}/leading_muon'
SAVE_PLOT = True

In [118]:
#Get data and POT
pfp = PFP.load(f'{DATA_DIR}/{FNAME}','pfp'
               ,prism_bins=PRISM_BINS
               ,momentum_bins=MOMENTUM_BINS
               ,costheta_bins=COSTHETA_BINS
               ,pot=NOM_POT)
slc = CAFSlice.load(f'{DATA_DIR}/{FNAME}','slice'
                    ,prism_bins=PRISM_BINS
                    ,pot=NOM_POT)
hdr = pd.read_hdf(f'{DATA_DIR}/{HDR_FNAME}',key='hdr')
SAMPLE_POT = hdr.pot.sum()

#Scale to nominal POT
print(f'--scaled by {NOM_POT/SAMPLE_POT:.2f}')
pfp.scale_to_pot(NOM_POT,SAMPLE_POT)
slc.scale_to_pot(NOM_POT,SAMPLE_POT)

--scaled by 6.93
--scaling to POT: 8.65e+18 -> 6.00e+19
--scaling to POT: 8.65e+18 -> 6.00e+19


In [119]:
CUTS = ['cosmic','fv','trk','has_muon']
pur,eff,f1 = slc.get_pur_eff_f1(['cosmic','fv','trk','has_muon'])
pur,eff,f1

(array([0.21441237, 0.50252181, 0.68067441, 0.69851779, 0.81824831]),
 array([1.        , 0.87595162, 0.7239054 , 0.7175484 , 0.58360706]),
 array([1.        , 0.63865546, 0.70162461, 0.70790522, 0.68129067]))

## 2 Apply cuts, get leading muon, assign binnings

In [120]:
slc_cut = slc.copy()
for cut in CUTS:
    slc_cut.apply_cut(cut)
#concat best_muon df with weights df
df = multicol_concat(slc_cut.data.best_muon,pd.DataFrame(slc_cut.data.genweight),axis=1)
df = multicol_concat(df,pd.DataFrame(slc_cut.data.truth.event_type),axis=1)
muon = PFP(df
              ,pot=NOM_POT
              ,prism_bins=PRISM_BINS
              ,momentum_bins=MOMENTUM_BINS
              ,costheta_bins=COSTHETA_BINS)
              

In [121]:
#Assign costheta bins - we do this after cuts to save some time - may postprocess this in the future
muon.assign_costheta_bins()
muon.assign_costheta_bins(key='truth.p.costheta',assign_key='truth.costheta_bin')

#Momentum bin
muon.assign_momentum_bins()
muon.assign_momentum_bins(key='truth.p.genp.tot',assign_key='truth.momentum_bin')

#Prism bin
muon.assign_prism_bins()
muon.assign_prism_bins(key='truth.p.prism_theta',assign_key='truth.prism_bin')

0it [00:00, ?it/s]

9it [00:04,  2.20it/s]
9it [00:04,  2.02it/s]
9it [00:03,  2.76it/s]
9it [00:03,  2.99it/s]
8it [00:03,  2.14it/s]
8it [00:03,  2.42it/s]


## 3 PRISM

In [122]:
weights = muon.data.genweight
bins = np.arange(-200,220,20)
for i,t in enumerate([True,False]):
    if t:
        pos = muon.data.truth.p.start
        fname = 'truth_prism'
    else:
        pos = muon.data.start
        fname = 'reco_prism'
    fig,ax=make_prism_plot(pos,weights=weights,bins=bins,cmap='Blues')
    ax = plot_volume_boundary(ax,lw=3,color='red',alpha=0.7)
    ax.set_xlim(-200,200)
    ax.set_ylim(-200,200)
    plotters.set_style(ax)
    ax.text(0.05,0.9,LABEL,transform=ax.transAxes,fontsize=16,color='darkred')
    #plotters.add_label(ax,LABEL,where='topright',color='white',size=12)
    if SAVE_PLOT:
        plotters.save_plot(fname,fig=fig,folder_name=PLOT_DIR)
        plt.close()

In [123]:
len(muon.data),len(slc_cut.data),len(slc.data)

(36913, 36913, 241376)

## 4 Single var distibutions

In [124]:
for i,dens in enumerate(['']):
    fig,ax = plt.subplots(figsize=(6,4))
    h = ax.hist([muon.data.p,muon.data.truth.p.genp.tot,],
                bins=np.arange(0,4,0.1),
                weights=[muon.data.genweight,muon.data.genweight,],
                histtype='step',
                lw=3,
                alpha=0.9,
                label=[f'Truth',
                f'Reco',],
                density=True if i == 1 else False, #set density
        )
    ax.legend()
    ax.set_xlabel(r'$p_\mu$ [GeV]')
    ax.set_ylabel('Normalized events' if i == 1 else 'Events')
    ax.set_title(f'Leading Muon Momentum ({round(muon.data.genweight.sum()):,} events)')

    plotters.set_style(ax,legend_loc='upper right')
    plotters.add_label(ax,LABEL,alpha=0.9,fontsize=12,color='gray',where='centerright')
    if SAVE_PLOT:
        plotters.save_plot(f'momentum',fig=fig,folder_name=PLOT_DIR)
        plt.close()

In [125]:
for i,dens in enumerate(['']):
    fig,ax = plt.subplots(figsize=(6,4))
    h = ax.hist([muon.data.costheta,muon.data.truth.p.costheta,],
                bins=np.arange(-1,1.1,0.1),
                weights=[muon.data.genweight,muon.data.genweight,],
                histtype='step',
                lw=3,
                alpha=0.9,
                label=[f'Truth',f'Reco',],
                density=True if i == 1 else False, #set density
        )
    ax.legend()
    ax.set_xlabel(r'$\cos\theta_\mu$')
    ax.set_ylabel('Normalized events' if i == 1 else 'Events')
    ax.set_title(fr'Leading Muon $\cos\theta$ ({round(muon.data.genweight.sum()):,} events)')

    plotters.set_style(ax,legend_loc='upper left')
    plotters.add_label(ax,LABEL,alpha=0.9,fontsize=12,color='gray',where='centerleft')
    if SAVE_PLOT:
        plotters.save_plot(f'momentum',fig=fig,folder_name=PLOT_DIR)
        plt.close()

## 5 Multivar distributions

In [126]:
#Make list of colors from colormap the same length as thetas
colors = plotters.get_colors('gnuplot2',len(PRISM_BINS)-1)

#Prism plot for muon kinematics - let's do this all at once to avoid double counting errors
for i,dens in enumerate(['','_dens']):
    #Figures forr angles
    fig_costheta,ax_costheta = plt.subplots(figsize=(8,6))
    
    #Figures for momenta
    fig_momentum,ax_momentum = plt.subplots(figsize=(8,6))
    
    #Make a list for repeated tasks
    axs = [ax_costheta,ax_momentum]
    
    #Labels for figures
    labels = [None]*(len(PRISM_BINS)-1)
    
    for j,_ in enumerate(PRISM_BINS):
        if PRISM_BINS[j] == PRISM_BINS[-1]: break #skip last bin to avoid range errors
        #Mask prism bins
        muon_inrange = muon.data[muon.data.prism_bin == j]
        
        #Set labels
        labels[j] = f'{round(PRISM_BINS[j],2)} < ' + r'$\theta_{PRISM}$'\
        + f' < {round(PRISM_BINS[j+1],2)} ({round(np.sum(muon_inrange.genweight)):,})'
        
        #Make histograms
        ax_costheta.hist(muon_inrange.costheta, #cos theta values
            bins=np.arange(-1,1.1,0.1),
            weights=muon_inrange.genweight,
            histtype='step',
            lw=2,
            alpha=0.9,
            label=labels[j],
            density=True if i == 1 else False, #set density
            linestyle='-' if j % 2 == 0 else '--', #alternate linestyle to help with visibility
            color=colors[j],
            )
        ax_momentum.hist(muon_inrange.p, #momentum values
            bins=np.arange(0,4,0.1),
            weights=muon_inrange.genweight,
            histtype='step',
            lw=2,
            alpha=0.9,
            label=labels[j],
            density=True if i == 1 else False, #set density
            color=colors[j],
            linestyle='-' if j % 2 == 0 else '--', #alternate linestyle to help with visibility
            )
    
    
    #Set xlabels
    ax_costheta.set_xlabel(r'$\cos\theta_\mu$')
    ax_momentum.set_xlabel(r'$p_\mu$ [GeV]')
    
    #Set labels
    plotters.add_label(ax_costheta,LABEL,alpha=0.8,fontsize=12,color='gray',where='bottomishleft')
    plotters.add_label(ax_momentum,LABEL,alpha=0.8,fontsize=12,color='gray',where='bottomishright')
    
    for k,ax in enumerate(axs):
        ax.set_ylabel('Normalized events' if i == 1 else 'Events')
        ax.legend()
    plotters.set_style(ax_costheta,legend_size=11,legend_loc='upper left')
    plotters.set_style(ax_momentum,legend_size=11,legend_loc='upper right')
    if SAVE_PLOT:
        plotters.save_plot(f'prism_muon_costheta{dens}',fig=fig_costheta,folder_name=PLOT_DIR)
        plotters.save_plot(f'prism_muon_momentum{dens}',fig=fig_momentum,folder_name=PLOT_DIR)
        plt.close('all')


In [127]:
for j,dens in enumerate(['','_dens']):
  fig,axs = plt.subplots(nrows=3,ncols=3,figsize=(15,12),sharex=True)
  for i,ax in enumerate(axs.flatten()):
    #Get muons within theta bins
    truth_muon_inrange = muon.data[(muon.data.truth.costheta_bin == i)]
    reco_muon_inrange = muon.data[(muon.data.costheta_bin == i)]
    
    #Get momenta
    ps_truth = truth_muon_inrange.truth.p.genp.tot
    ps_reco = reco_muon_inrange.p
    
    #Get weights
    weights_truth = truth_muon_inrange.genweight
    weights_reco = reco_muon_inrange.genweight
    
    #Get number of events from weights
    truth_inrange_count = np.sum(weights_truth)
    reco_inrange_count = np.sum(weights_reco)
    
    ax.hist([ps_truth,ps_reco],
            bins=np.arange(0,4,0.1),
            weights=[weights_truth,weights_reco],
            histtype='step',
            lw=2,
            label=[f'Truth ({round(truth_inrange_count):,})', 
                  f'Reco ({round(reco_inrange_count):,})'],
            density=True if j == 1 else False)
    
    plotters.set_style(ax)
    plotters.add_label(ax,f'{LABEL}\n{COSTHETA_BINS[i]:.2f} < $\cos\\theta_\mu$ < {COSTHETA_BINS[i+1]:.2f}',fontsize=12,alpha=0.9,where='centerright')
    ax.legend()
  axs[2,1].set_xlabel(r'$p_\mu$ [GeV]',fontsize=20)
  axs[0,1].set_title(rf'{round(muon.data.genweight.sum()):,} muons',fontsize=25)
  axs[1,0].set_ylabel('Normalized events' if j == 1 else 'Events',fontsize=20)
  if SAVE_PLOT:
    plotters.save_plot(f'momentum_mu_theta{dens}',fig=fig,folder_name=PLOT_DIR)
    plt.close()

In [128]:
for j,dens in enumerate(['','_dens']):
  fig,axs = plt.subplots(nrows=3,ncols=3,figsize=(15,12),sharex=True)
  for i,ax in enumerate(axs.flatten()):
    #Get muons within theta bins
    truth_muon_inrange = muon.data[(muon.data.truth.momentum_bin == i)]
    reco_muon_inrange = muon.data[(muon.data.momentum_bin == i)]
    
    #Get momenta
    ps_truth = truth_muon_inrange.truth.p.costheta
    ps_reco = reco_muon_inrange.costheta
    
    #Get weights
    weights_truth = truth_muon_inrange.genweight
    weights_reco = reco_muon_inrange.genweight
    
    #Get number of events from weights
    truth_inrange_count = np.sum(weights_truth)
    reco_inrange_count = np.sum(weights_reco)
    
    ax.hist([ps_truth,ps_reco],
            bins=np.arange(-1,1.1,0.1),
            weights=[weights_truth,weights_reco],
            histtype='step',
            lw=2,
            label=[f'Truth ({round(truth_inrange_count):,})', 
                  f'Reco ({round(reco_inrange_count):,})'],
            density=True if j == 1 else False)
    
    plotters.set_style(ax)
    plotters.add_label(ax,f'{LABEL}\n{MOMENTUM_BINS[i]:.2f} < $p_\mu$ < {MOMENTUM_BINS[i+1]:.2f}',fontsize=12,alpha=0.9,where='centerleft')
    ax.legend()
  axs[2,1].set_xlabel(r'$\cos\theta_\mu$',fontsize=20)
  axs[0,1].set_title(rf'{round(muon.data.genweight.sum()):,} muons',fontsize=25)
  axs[1,0].set_ylabel('Normalized events' if j == 1 else 'Events',fontsize=20)
  if SAVE_PLOT:
    plotters.save_plot(f'theta_mu_momentum{dens}',fig=fig,folder_name=PLOT_DIR)
    plt.close()

## 6 2D histograms w uncertainties

In [129]:
true_numucc = (muon.data.event_type == 0)

In [130]:
#Energy
no_nan = ~muon.data.energy.isna() & ~muon.data.truth.p.startE.isna()
for norm in [True,False]:
    for use_numucc in [True,False]:
        fname = 'energy'
        title = ''
        if use_numucc:
            fname += '_numucc'
            title += r'True $\nu_\mu$CC'
            mask = true_numucc & no_nan
        else:
            mask = no_nan
        x = muon.data.energy[mask]
        y = muon.data.truth.p.startE[mask]
        bins = np.arange(0,1.55,0.05)
        if norm:
            fname += '_norm'
            norm = LogNorm()
            title += ' (Log Scale)'
        else:
            norm = None
        fig,(ax,ax2) = makeplot.plot_hist2d_frac_err(x,y,xlabel=r'$E_{reco}$ [GeV]',ylabel=r'$E_{true}$ [GeV]',bins=bins,cmap='Blues',plot_line=True,norm=norm)
        ax.set_title(title)
        plotters.set_style(ax)
        plotters.add_label(ax,LABEL,where='topleft',color='red',size=12)
        if SAVE_PLOT:
            plotters.save_plot(fname,fig=fig,folder_name=f'{PLOT_DIR}/2dhists')
            plt.close()

In [131]:
#Costheta
no_nan = ~muon.data.costheta.isna() & ~muon.data.truth.p.costheta.isna()
for norm in [True,False]:
    for use_numucc in [True,False]:
        fname = 'costheta'
        title = ''
        if use_numucc:
            mask = true_numucc & no_nan
            fname += '_numucc'
            title += r'True $\nu_\mu$CC'
        else:
            mask = no_nan
        if norm:
            norm = LogNorm()
            fname += '_norm'
            title += ' (Log Scale)'
        else:
            norm = None
        x = muon.data.costheta[mask]
        y = muon.data.truth.p.costheta[mask]
        bins = np.arange(-1,1.1,0.1)
        fig,(ax,ax2) = makeplot.plot_hist2d_frac_err(x,y,xlabel=r'$\cos\theta_{reco}$',ylabel=r'$\cos\theta_{true}$',bins=bins,cmap='Blues',plot_line=True,norm=norm)
        ax.set_title(title)
        plotters.set_style(ax)
        plotters.add_label(ax,LABEL,where='topleft',color='red',size=12)
        if SAVE_PLOT:
            plotters.save_plot(fname,fig=fig,folder_name=f'{PLOT_DIR}/2dhists')
            plt.close()
        bins = np.arange(0.5,1.025,0.025)
        fig,(ax,ax2) = makeplot.plot_hist2d_frac_err(x,y,xlabel=r'$\cos\theta_{reco}$',ylabel=r'$\cos\theta_{true}$',bins=bins,cmap='Blues',plot_line=True,norm=norm)
        ax.set_title(title)
        plotters.set_style(ax)
        plotters.add_label(ax,LABEL,where='topleft',color='red',size=12)
        if SAVE_PLOT:
            plotters.save_plot(f'{fname}_zoom',fig=fig,folder_name=f'{PLOT_DIR}/2dhists')
            plt.close()